## Dataset de revisiones físicas

### Limpieza inicial

In [1]:
import pandas as pd
import datetime
df_check = pd.read_csv('../Dataset/datos_check.csv')


In [2]:
df_check

,negocio_id,fecha
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."
...,...,...
161945,zzvlwkcNR1CCqOPXwuvz2A,"2017-05-06 20:05:15, 2017-05-12 22:37:03, 2017..."
161946,zzwaS0xn1MVEPEf0hNLjew,"2010-02-16 02:09:56, 2010-07-05 05:40:48, 2010..."
161947,zzwhN7x37nyjP0ZM8oiHmw,"2016-03-06 13:27:02, 2016-03-09 00:41:53, 2016..."
161948,zzwicjPC9g246MK2M1ZFBA,"2012-09-22 00:26:15, 2012-09-23 20:12:00, 2012..."


### Búsqueda de valores nulos

In [3]:
print("¿Hay valores nulos?\n", pd.isnull(df_check).any(), "\n")
print("¿Cuantos?\n", pd.isnull(df_check).sum(), "\n")

¿Hay valores nulos?
 negocio_id    False
fecha         False
dtype: bool 

¿Cuantos?
 negocio_id    0
fecha         0
dtype: int64 



### Comprobación sintáctica de la fecha

No se imprime ningún error por lo que, aparentemente, todas las fechas son sintácticamente correctas.

In [4]:
import re
pattern = re.compile("[\s]?[0-9][0-9][0-9][0-9][-][0-1][0-9][-][0-3][0-9]\s[0-2][0-9][:][0-5][0-9][:][0-5][0-9]")


def preprocesa(x):
    listaFechas = x.split(",")
    for date in listaFechas:
        if(pattern.match(date) == None):
            print("date not matching: " + str(x))
            return True
    return False

df_check['fecha'].apply(lambda x: preprocesa(x))

0         False
1         False
2         False
3         False
4         False
          ...  
161945    False
161946    False
161947    False
161948    False
161949    False
Name: fecha, Length: 161950, dtype: bool

### IMPORTANTE 
A continuacion vamos a tratar de sacar informacion del los datos. Debido a que solo hay dos columnas una identificadora del negocio y otra que contiene una listas de fechas con las revisiones de cada negocio, la mayoria de informacion se encuentra en esta columna y es la que vamos a tratar. Se tendran dos enfoques:

-Generar un dataframe con el numero de revisiones que tiene cada negocio y el intervalo de las revisiones. Centrandonos mas en un solo negocio.

-Generar un dataframe separando las listas de los negocios de tuplas negocio/revision. Y posteriormente separar esa fecha de negocio en año mes dia hora minuto y dia de la semana para tratar de generar graficos agrupando por esos atributos y ver si de manera global las revisiones depende de la fecha en la que se hacen.


Calculamos el numero de revisiones que tiene cada negocio y creamos una columna nueva para cada negocio. Con esto podremos determinar que negocios son los que mas revisiones tienen y si llegan a tener demasiadas investigar a que se debe este fenomeno.

In [5]:
def n_revisiones(x):
    listaFechas = x.split(",")
    return len(listaFechas)
df_check['num_revisiones']=df_check['fecha'].apply(lambda x:n_revisiones(x))
 

Convertimos el formato string de la columna fecha en datetime para poder operar con el posteriormente con ellas y manejarlas con facilidad

In [6]:
def pasarfechas(x):
    fechas=[]
    listaFechas = x.split(",")
    for i in listaFechas:
        date=datetime.datetime.strptime(i.strip(),'%Y-%m-%d %H:%M:%S')
        fechas.append(date)
    return fechas
df_check['fechas']=df_check['fecha'].apply(lambda x:pasarfechas(x))


In [7]:
df_mediarevisiones=df_check
def media(x):
    colum=0
    aux=[None]*(len(x)-1)
    if(len(x)==1):
        return 0
    while colum < (len(x)-1):
        resultado=(x[colum]-x[colum+1])
        aux[colum]=resultado
        colum=colum+1
    i=0
    suma=aux[0]-aux[0]
    while i < (len(x)-1):
        suma=suma+aux[i]
        i=i+1
    media=-suma/(len(x)-1)
    del aux
    return media
df_mediarevisiones['media']=df_mediarevisiones['fechas'].apply(lambda x:media(x))
df_mediarevisiones=df_mediarevisiones.drop(['fecha'],axis=1)


In [8]:
df_mediarevisiones.to_pickle("../Dataset/df_checksmedias_revisiones.pkl")

Sabiendo que la gran cantidad de revisiones que puede llegar a tener un solo negocio se han tomado medidas para poder sacar cada tupla revision/negocio

In [9]:
df_check.describe()

,num_revisiones
count,161950.000000
mean,117.870627
std,733.721561
min,1.000000
25%,5.000000
50%,16.000000
75%,65.000000
max,143061.000000


### Importante
Acontinuacion se quieren separar las listas de revisiones de cada negocio en tuplas para generar el segundo dataframe.
Debido a que no se puede tratar entero dataframe check este se tiene que partir en distintos dataframes que separen las revisiones y luego juntarlas de nuevo en otro fichero. Los dataframes en los que se divide son:

-**Df_Checking_menor**: Con rangos de numero de revisiones de 0 a 69.

-**df_checking_medio**: Con rangos de numero de revisiones de 70 a 499

-**df_checking_alto_medio**: Con rangos de numero de revisiones de 500 a 2999.

-**df_checking_alto_alto**: Con rangos de numero de revisiones de 3000 a 999999

-**df_checking_demasiado**: Con rangos de numero de revisiones de 100000 al maximo

In [11]:
df_checking=df_check.drop(df_check[df_check.num_revisiones>70].index)
df_checking=df_checking.drop(['num_revisiones','fecha','media'],axis=1)
fecha=df_checking['fechas'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index','value']].set_index('index')
df_checking=pd.merge(fecha,df_checking[['negocio_id']],left_index=True,right_index=True).rename(columns={'value_x': 'negocio_id', 'value_y' :'fechas'})
df_checking.to_pickle("../Dataset/dfcheckmenor.pkl")

In [12]:

df_checking=df_check.drop(df_check[df_check.num_revisiones>500].index)
df_checking=df_checking.drop(df_checking[df_checking.num_revisiones<70].index)
df_checking=df_checking.drop(['num_revisiones','fecha','media','media'],axis=1)
fecha=df_checking['fechas'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index','value']].set_index('index')
df_checking=pd.merge(fecha,df_checking[['negocio_id']],left_index=True,right_index=True).rename(columns={'value_x': 'negocio_id', 'value_y' :'fechas'})
df_checking.to_pickle("../Dataset/dfcheckmedio.pkl")

In [13]:

df_checking=df_check.drop(df_check[df_check.num_revisiones>3000].index)
df_checking=df_checking.drop(df_checking[df_checking.num_revisiones<500].index)
df_checking=df_checking.drop(['num_revisiones','fecha','media'],axis=1)
fecha=df_checking['fechas'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index','value']].set_index('index')
df_checking=pd.merge(fecha,df_checking[['negocio_id']],left_index=True,right_index=True).rename(columns={'value_x': 'negocio_id', 'value_y' :'fechas'})
df_checking.to_pickle("../Dataset/dfcheckaltomedio.pkl")

In [14]:
df_checking=df_check.drop(df_check[df_check.num_revisiones>100000].index)
df_checking=df_checking.drop(df_checking[df_checking.num_revisiones<3000].index)
df_checking=df_checking.drop(['num_revisiones','fecha','media'],axis=1)
fecha=df_checking['fechas'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index','value']].set_index('index')
df_checking=pd.merge(fecha,df_checking[['negocio_id']],left_index=True,right_index=True).rename(columns={'value_x': 'negocio_id', 'value_y' :'fechas'})
df_checking.to_pickle("../Dataset/dfcheckaltoalto.pkl")

In [15]:
df_checking=df_check.drop(df_check[df_check.num_revisiones<100000].index)
df_checking=df_checking.drop(['num_revisiones','fecha','media'],axis=1)
fecha=df_checking['fechas'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index','value']].set_index('index')
df_checking=pd.merge(fecha,df_checking[['negocio_id']],left_index=True,right_index=True).rename(columns={'value_x': 'negocio_id', 'value_y' :'fechas'})
df_checking.to_pickle("../Dataset/dfcheckdemasiado.pkl")

### Extracion de los ficheros PKL 

Posteriormente cada uno de los dataframes calculados se guarda en un fichero pkl para su posterior union.


In [16]:
dfcheckmenor= pd.read_pickle("../Dataset/dfcheckmenor.pkl")
dfcheckmedio= pd.read_pickle("../Dataset/dfcheckmedio.pkl")
dfcheckaltomedio= pd.read_pickle("../Dataset/dfcheckaltomedio.pkl")
dfcheckaltoalto= pd.read_pickle("../Dataset/dfcheckaltoalto.pkl")
dfcheckdemasiado= pd.read_pickle("../Dataset/dfcheckdemasiado.pkl")

In [17]:
a=dfcheckmenor.append(dfcheckmedio)
b=a.append(dfcheckaltomedio)
a=b.append(dfcheckaltoalto)
b=a.append(dfcheckdemasiado)
b=b.sort_index()
b=b.rename(columns={"value":'fecha'})

In [18]:
b['anyo'] = pd.DatetimeIndex(b['fecha']).year
b['mes'] = pd.DatetimeIndex(b['fecha']).month
b['dia'] = pd.DatetimeIndex(b['fecha']).day
b['hora'] = pd.DatetimeIndex(b['fecha']).hour
b['minuto'] = pd.DatetimeIndex(b['fecha']).minute

In [19]:
b.to_pickle("../Dataset/df_check_timestamp_separado.pkl")